In [74]:
import mlflow

import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [75]:
wind_farm_df = pd.read_csv("./datasets/windfarm_data.csv", index_col=0)

In [76]:
wind_farm_df.index

Index(['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04', '2014-01-05',
       '2014-01-06', '2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10',
       ...
       '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-25', '2020-12-26',
       '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31'],
      dtype='object', length=2555)

In [77]:
wind_farm_df.tail()

,temperature_00,wind_direction_00,wind_speed_00,temperature_08,wind_direction_08,wind_speed_08,temperature_16,wind_direction_16,wind_speed_16,power
2020-12-27,7.123225,103.176630,8.133746,6.454002,107.793220,6.326991,7.219884,119.070526,3.062219,2621.4760
2020-12-28,5.376270,118.084330,5.558247,8.118839,116.193535,8.565966,9.307176,120.264430,11.993913,5423.6250
2020-12-29,8.593436,115.432590,12.181850,8.587968,112.931360,11.970859,8.956771,110.161095,11.301485,9132.1150
2020-12-30,8.069033,103.169685,9.983466,7.930485,106.045510,6.381556,8.228901,111.602160,4.087358,3667.9927
2020-12-31,6.425237,115.713690,3.755484,7.935767,109.338030,5.126760,11.186373,105.873110,5.630923,858.9223


In [78]:
wind_farm_df.describe()


,temperature_00,wind_direction_00,wind_speed_00,temperature_08,wind_direction_08,wind_speed_08,temperature_16,wind_direction_16,wind_speed_16,power
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,9.613588,198.381353,5.001413,12.827786,191.538394,5.136628,13.991459,202.051406,6.072578,2424.978880
std,5.187017,81.767411,2.940271,6.536972,83.572022,3.016934,6.862011,88.747683,2.585946,2035.838491
min,-6.172769,14.527682,0.232208,-3.989666,14.087781,0.519220,-4.087519,5.659133,0.561177,207.065570
25%,6.228526,111.309685,2.863503,7.932104,108.053078,2.897562,8.576865,111.450595,4.299781,947.100430
50%,8.688278,214.461780,4.265017,11.178736,174.028730,4.356376,12.392289,245.210940,5.785738,1674.430900
75%,13.045183,276.894330,6.263791,17.898135,281.361380,6.745475,19.687266,275.937150,7.550025,3274.856450
max,27.039269,352.764130,19.162184,30.118828,351.428300,23.394577,32.521378,349.589100,18.859266,10774.853000


In [72]:
wind_farm_df["2015-01-01":"2018-01-01"].tail()

,temperature_00,wind_direction_00,wind_speed_00,temperature_08,wind_direction_08,wind_speed_08,temperature_16,wind_direction_16,wind_speed_16,power
2017-12-28,6.505444,21.090723,1.728905,7.756409,49.636170,1.844953,8.003177,141.539140,1.931224,222.44484
2017-12-29,6.150245,247.736480,8.716457,2.962454,275.812500,10.803079,2.458707,283.338750,9.119889,6943.74460
2017-12-30,1.804310,264.925870,8.026327,3.297666,277.995540,4.768021,2.136792,276.178250,2.080063,2222.43020
2017-12-31,0.295758,108.385796,4.245270,2.566788,99.842730,8.125836,4.871737,101.659065,8.688424,4079.36160
2018-01-01,4.290002,99.383450,11.876492,5.606760,99.889145,11.322479,6.574509,106.049570,9.184392,8226.33400


In [79]:
def get_training_data():
    training_data = wind_farm_df["2014-01-01":"2017-12-31"]
    X = training_data.drop(columns="power")
    y = training_data["power"]
    return X, y

In [80]:
def get_validation_data():
    training_data = wind_farm_df["2018-01-01":"2019-12-31"]
    X = training_data.drop(columns="power")
    y = training_data["power"]
    return X, y

In [81]:
def get_weather_and_forecast(weather_date):
    format_date = lambda pd_date: pd_date.date().strftime("%Y-%m-%d")
    today = pd.Timestamp(weather_date).normalize()
    week_ago = today - pd.Timedelta(days=5)
    week_later = today + pd.Timedelta(days=5)
    
    past_power_output = pd.DataFrame(wind_farm_df)[format_date(week_ago):format_date(week_later)]
    weather_and_forecast = pd.DataFrame(wind_farm_df)[format_date(week_ago):format_date(week_later)]
    
    if len(weather_and_forecast) < 10:
        past_power_output = pd.DataFrame(wind_farm_df).iloc[-10:5]
        weather_and_forecast = pd.DataFrame(wind_farm_df).iloc[-10:1]
        
    return weather_and_forecast.drop(columns="power"), past_power_output["power"]


In [83]:
# Testers

# pd.Timestamp('2020-1-1').normalize()
# w1, p1 = get_weather_and_forecast('2020-1-1')
# w1
# p1
X_train, y_train = get_training_data()

In [84]:
# Train Keras model

def train_keras_model():
    model = Sequential()
    model.add(Dense(100, input_shape=(X_train.shape[-1],), activation='relu', name="hidden_layer"))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.fit(X_train, y_train, epochs=25, batch_size=64, validation_split=0.2)
    return model

In [86]:
import mlflow
import mlflow.keras
import mlflow.tensorflow

X_train, y_train = get_training_data()

with mlflow.start_run():
    #Automatically capture model's parameters, metrics, artifacts,
    # and source code with the autolog() function
    
    mlflow.tensorflow.autolog()
    
    train_keras_model()
    
    run_id = mlflow.active_run().info.run_id
    
    

2023/09/10 17:14:35 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'pandas.core.frame.DataFrame'>. Dataset logging skipped.


Epoch 1/25
 1/19 [>.............................] - ETA: 15s - loss: 8183865.0000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_train_batch_end` time: 0.0760s). Check your callbacks.


19/19 [==============================] - 4s 179ms/step - loss: 10347726.0000 - val_loss: 7843145.5000
Epoch 2/25
19/19 [==============================] - 2s 117ms/step - loss: 9884447.0000 - val_loss: 7396907.0000
Epoch 3/25
19/19 [==============================] - 1s 50ms/step - loss: 9375887.0000 - val_loss: 6897687.5000
Epoch 4/25
19/19 [==============================] - 1s 51ms/step - loss: 8789144.0000 - val_loss: 6344701.5000
Epoch 5/25
19/19 [==============================] - 1s 42ms/step - loss: 8148389.0000 - val_loss: 5789598.5000
Epoch 6/25
19/19 [==============================] - 1s 72ms/step - loss: 7497087.0000 - val_loss: 5252006.5000
Epoch 7/25
19/19 [==============================] - 1s 47ms/step - loss: 6890807.5000 - val_loss: 4812088.0000
Epoch 8/25
19/19 [==============================] - 1s 62ms/step - loss: 6352429.5000 - val_loss: 4526761.5000
Epoch 9/25
19/19 [==============================] - 1s 38ms/step - loss: 5969111.5000 - val_loss: 4365381.5000
Epoch 10/

2023/09/10 17:15:01 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: Cannot log input example or model signature for input with type <class 'pandas.core.frame.DataFrame'>. TensorFlow Keras autologging can only log input examples and model signatures for the following input types: numpy.ndarray, dict[string -> numpy.ndarray], tensorflow.keras.utils.Sequence, and tensorflow.data.Dataset (TensorFlow >= 2.1.0 required)
2023/09/10 17:15:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\coool\AppData\Local\Temp\tmpztvr1x7u\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\coool\AppData\Local\Temp\tmpztvr1x7u\model\data\model\assets


In [87]:
model_name = "power-forecasting-model"

In [89]:
run_id

'014acc5e254448feb9fe14784c5fae6d'